In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
path = '/kaggle/input/itstep-exam-1/data.csv'

In [ ]:
data = pd.read_csv(path, index_col="Unnamed: 0")
data.head()

In [ ]:

data.info()
data.describe(include='all')

In [ ]:
data.describe()

In [ ]:
missing_values = data.isnull().sum()

# Виведення результату
print(missing_values)

# Загальна кількість пропущених значень
total_missing = missing_values.sum()
print(f"Загальна кількість пропущених значень: {total_missing}")

In [ ]:
data['Bed Grade'] = data['Bed Grade'].fillna(data['Bed Grade'].median())
data['City_Code_Patient'] = data['City_Code_Patient'].fillna(-1)


In [ ]:
data = data.drop(columns=['doctorid']) 
data = data.drop(columns=['case_id']) 
data

In [ ]:
# Виділення числових та категоріальних ознак
numerical_cols = data.select_dtypes(include=[np.number]).columns
categorical_cols = data.select_dtypes(exclude=[np.number]).columns

print("Числові ознаки:", numerical_cols)
print("Категоріальні ознаки:", categorical_cols)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# Кількість стовпчиків для гістограм
num_bins = int(np.sqrt(len(data)))

# Словник для опису гістограм
hist_descriptions = {}

# Побудова гістограм для числових ознак
for col in numerical_cols:
    plt.figure(figsize=(10, 6))
    
    # Вибір непустих значень
    col_data = data[col].dropna()
    
    # Створення гістограми
    counts, bin_edges = np.histogram(col_data, bins=num_bins)
    plt.hist(col_data, bins=num_bins, edgecolor='k', alpha=0.7)
    plt.title(f'Гістограма для {col}', fontsize=14)
    plt.xlabel(col, fontsize=12)
    plt.ylabel('Частота', fontsize=12)
    plt.grid(True)
    plt.show()
    
    # Аналіз гістограми
    # Визначення викидів
    Q1 = col_data.quantile(0.25)
    Q3 = col_data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = col_data[(col_data < lower_bound) | (col_data > upper_bound)]
    
    # Визначення кількості піків
    peaks, _ = find_peaks(counts)
    num_peaks = len(peaks)
    
    # Зміщення
    skewness = col_data.skew()
    
    hist_descriptions[col] = {
        "Кількість викидів": len(outliers),
        "Кількість піків": num_peaks,
        "Зміщення": "Вліво" if skewness < 0 else ("Вправо" if skewness > 0 else "Симетрична")
    }

# Виведення опису гістограм
for col, desc in hist_descriptions.items():
    print(f"\nОпис гістограми для {col}:")
    for key, value in desc.items():
        print(f"- {key}: {value}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Словник для опису ящиків з вусами
boxplot_descriptions = {}

# Побудова ящиків з вусами для числових ознак
for col in numerical_cols:
    plt.figure(figsize=(10, 6))
    
    # Вибір непустих значень
    col_data = data[col].dropna()
    
    # Побудова ящика з вусами
    plt.boxplot(col_data, vert=False)
    plt.title(f'Ящик з вусами для {col}', fontsize=14)
    plt.xlabel(col, fontsize=12)
    plt.grid(True)
    plt.show()
    
    # Опис ящика
    Q1 = col_data.quantile(0.25)
    Q3 = col_data.quantile(0.75)
    median = col_data.median()
    IQR = Q3 - Q1
    
    # Визначення меж вусів
    lower_whisker = Q1 - 1.5 * IQR
    upper_whisker = Q3 + 1.5 * IQR
    
    # Визначення викидів
    outliers = col_data[(col_data < lower_whisker) | (col_data > upper_whisker)]
    
    # Зміщення ящика
    box_skew = "Невизначене"
    if median < Q1:
        box_skew = "Вліво"
    elif median > Q3:
        box_skew = "Вправо"
    else:
        box_skew = "Симетричне"
    
    boxplot_descriptions[col] = {
        "25% - 50% - 75% діапазони": f"{Q1:.2f} - {median:.2f} - {Q3:.2f}",
        "100% діапазон": f"{col_data.min():.2f} - {col_data.max():.2f}",
        "Кількість викидів": len(outliers),
        "Зміщення": box_skew
    }

# Виведення опису для кожного ящика з вусами
for col, desc in boxplot_descriptions.items():
    print(f"\nОпис ящика з вусами для {col}:")
    for key, value in desc.items():
        print(f"- {key}: {value}")


In [ ]:
# Видаляємо викиди
data = data[data['Visitors with Patient'] < 26]
data['Visitors with Patient'].plot(kind='box', vert=False)

In [ ]:
data = data[data['Available Extra Rooms in Hospital'] < 15]
data['Available Extra Rooms in Hospital'].plot(kind='box', vert=False)

In [ ]:
data = data[data['City_Code_Patient'] < 25]
data['City_Code_Patient'].plot(kind='box', vert=False)

In [ ]:
data = data[data['Admission_Deposit'] < 8000]
data['Admission_Deposit'].plot(kind='box', vert=False)

In [ ]:
# Виведення гістограм для категоріальних ознак
for col in categorical_cols:
    plt.figure(figsize=(10, 6))
    sns.countplot(x=data[col], palette="Set2")
    plt.title(f'Гістограма для {col}', fontsize=14)
    plt.xlabel(col, fontsize=12)
    plt.ylabel('Кількість', fontsize=12)
    plt.grid(True)
    plt.show()

In [ ]:
# Групові ящики з вусами для числових ознак, залежно від категоріальних ознак
for num_col in numerical_cols:
    for cat_col in categorical_cols:
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=cat_col, y=num_col, data=data, palette="Set3")
        plt.title(f'Ящик з вусами для {num_col} за категорією {cat_col}', fontsize=14)
        plt.xlabel(cat_col, fontsize=12)
        plt.ylabel(num_col, fontsize=12)
        plt.grid(True)
        plt.show()

        # Опис залежності
        if data.groupby(cat_col)[num_col].mean().std() > 0:
            print(f"Є залежність між {num_col} та {cat_col}: числова ознака змінюється залежно від категорії {cat_col}")
        else:
            print(f"Немає значної залежності між {num_col} та {cat_col}")

In [ ]:
# Точкові діаграми для числових ознак
for col1 in numerical_cols:
    for col2 in numerical_cols:
        if col1 != col2:
            plt.figure(figsize=(10, 6))
            sns.scatterplot(x=data[col1], y=data[col2], alpha=0.7)
            plt.title(f'Точкова діаграма для {col1} та {col2}', fontsize=14)
            plt.xlabel(col1, fontsize=12)
            plt.ylabel(col2, fontsize=12)
            plt.grid(True)
            plt.show()
            
            # Визначення залежності
            corr = data[col1].corr(data[col2])
            if abs(corr) > 0.5:
                print(f"Є сильна залежність між {col1} та {col2} (коефіцієнт кореляції: {corr:.2f})")
            else:
                print(f"Немає сильної залежності між {col1} та {col2}")

In [ ]:
data

In [ ]:
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int', 'float']).columns

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import optuna
import joblib

# Виділення категоріальних і числових ознак
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int', 'float']).columns

# Препроцесор
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'
)

# Розділення даних
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Оптимізація з використанням Optuna
def objective(trial):
    # Гіперпараметри для RandomForest
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)

    # Побудова пайплайна
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        ))
    ])

    # Тренування моделі
    model.fit(X_train, y_train)

    # Прогнозування
    y_pred = model.predict(X_test)

    # Метрика F1 для оцінки
    f1 = f1_score(y_test, y_pred, average='weighted')
    return f1

# Запуск Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Кращі гіперпараметри
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

# Модель уже оптимізована під час Optuna
final_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        random_state=42
    ))
])

# Повторне навчання з оптимальними параметрами
final_model.fit(X_train, y_train)

# Прогнозування
y_pred = final_model.predict(X_test)

# Оцінка якості моделі
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Виведення результатів
print(f"Accuracy: {accuracy}")
print(f"F1-Score: {f1}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Збереження моделі
joblib.dump(final_model, 'optimized_random_forest_model.joblib')


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, classification_report
import numpy as np

# Прогнозування для тестових даних
y_pred = final_model.predict(X_test)

# Отримання classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Вибір класів
class_names = report.keys()

# Для кожного класу будуємо precision-recall curve
plt.figure(figsize=(8, 6))
for class_name in class_names:
    if class_name not in ['accuracy', 'macro avg', 'weighted avg']:
        precision, recall, _ = precision_recall_curve(
            (y_test == class_name).astype(int), 
            (y_pred == class_name).astype(int)
        )
        plt.plot(recall, precision, label=f'Class {class_name}')
        
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for each class')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Обчислення матриці сплутаних результатів
cm = confusion_matrix(y_test, y_pred)

# Візуалізація матриці сплутаних результатів
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# Бінаризація класів
y_test_bin = label_binarize(y_test, classes=np.unique(y_test))
y_pred_prob = final_model.predict_proba(X_test)

# Побудова ROC для кожного класу
plt.figure(figsize=(8, 6))
for i in range(len(np.unique(y_test))):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_pred_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for each class')
plt.legend(loc='lower right')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import optuna
import joblib

# Перевірка наявності цільової змінної в датасеті
if 'Stay' not in data.columns:
    raise ValueError("Column 'Stay' not found in the dataset.")

# 1. Розділення даних на ознаки (X) та цільову змінну (y)
X = data.drop(columns=['Stay'])  # Всі стовпці, крім цільової змінної
y = data['Stay']  # Текстові категорії

# 2. Розділення на тренувальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Створення препроцесора для категоріальних змінних
categorical_features = X.select_dtypes(include=['object']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ], 
    remainder='passthrough'
)

# 4. Оптимізація гіперпараметрів за допомогою Optuna
def objective(trial):
    # Гіперпараметри RandomForest
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    
    # Пайплайн моделі
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            random_state=42
        ))
    ])
    
    # Тренування моделі
    model.fit(X_train, y_train)
    
    # Прогнозування
    y_pred = model.predict(X_test)
    
    # Метрика F1-Score
    f1 = f1_score(y_test, y_pred, average='weighted')
    return f1

# Оптимізація за допомогою Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Найкращі параметри
print('Best hyperparameters: ', study.best_params)

# 5. Побудова моделі з найкращими параметрами
best_params = study.best_params
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        random_state=42
    ))
])

# Тренування моделі
model.fit(X_train, y_train)

# Прогнозування на тестовій вибірці
y_pred = model.predict(X_test)

# 6. Оцінка моделі
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Результати
print(f'Accuracy: {accuracy}')
print(f'F1-Score: {f1}')
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

# 7. Збереження моделі
joblib.dump(model, 'random_forest_model.joblib')
print(f'Модель збережено у файл "random_forest_model.joblib".')


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
# from sklearn.preprocessing import LabelEncoder
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
# import optuna
# import joblib

# # 1. Обробка категоріальної цільової змінної
# # Використовуємо Label Encoding для категоріальних ознак "Stay"
# le = LabelEncoder()
# data['Stay_encoded'] = le.fit_transform(data['Stay'])

# # 2. Розділення даних на ознаки (X) та цільову змінну (y)
# X = data.drop(columns=['Stay', 'Stay_encoded'])  # Всі стовпці, крім 'Stay'
# y = data['Stay_encoded']  # Цільова змінна

# # 3. Розділення на тренувальну та тестову вибірки
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # 4. Створення препроцесора для категоріальних змінних
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('cat', OneHotEncoder(), X.select_dtypes(include=['object']).columns)  # Перетворення категоріальних змінних
#     ], 
#     remainder='passthrough'  # Оставити інші числові ознаки без змін
# )

# # 5. Оптимізація гіперпараметрів за допомогою Optuna
# def objective(trial):
#     # Створення моделі RandomForest з гіперпараметрами, що оптимізуються
#     n_estimators = trial.suggest_int('n_estimators', 50, 200)
#     max_depth = trial.suggest_int('max_depth', 3, 20)
#     min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    
#     # Пайплайн для препроцесування і тренування моделі
#     model = Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('classifier', RandomForestClassifier(
#             n_estimators=n_estimators,
#             max_depth=max_depth,
#             min_samples_split=min_samples_split,
#             random_state=42
#         ))
#     ])
    
#     # Тренування моделі
#     model.fit(X_train, y_train)
    
#     # Прогнозування на тестових даних
#     y_pred = model.predict(X_test)
    
#     # Оцінка метрики F1-Score
#     f1 = f1_score(y_test, y_pred, average='weighted')
#     return f1  # Оптимізуємо F1-Score

# # Пошук кращих гіперпараметрів за допомогою Optuna
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=50)

# # Кращі гіперпараметри
# print('Best hyperparameters: ', study.best_params)

# # 6. Побудова моделі з оптимальними гіперпараметрами
# best_params = study.best_params
# model = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('classifier', RandomForestClassifier(
#         n_estimators=best_params['n_estimators'],
#         max_depth=best_params['max_depth'],
#         min_samples_split=best_params['min_samples_split'],
#         random_state=42
#     ))
# ])

# # Тренування моделі з оптимальними параметрами
# model.fit(X_train, y_train)

# # 7. Прогнозування на тестових даних
# y_pred = model.predict(X_test)

# # 8. Оцінка якості моделі
# accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred, average='weighted')

# # Виведення метрик
# print(f'Accuracy: {accuracy}')
# print(f'F1-Score: {f1}')
# print('Classification Report:')
# print(classification_report(y_test, y_pred))
# print('Confusion Matrix:')
# print(confusion_matrix(y_test, y_pred))

# # 9. Збереження моделі
# joblib.dump(model, 'random_forest_model.joblib')

# # 10. Виведення метрик
# print(f'Оптимальні гіперпараметри: {best_params}')
